# IMAGES FROM  INATURALIST NZ AS PER EXOTIC LIST

    - This program downloads images from the inaturalist.nz as per the exotic species list.
    - These images were required for testing the machine learning model for selecting visually similar images.
    - Input : Any file which contains names of 1500 exotic speceis list
    - output : images downloaded in a new folder with names of images appended as abc_0.jpeg, abc_1.jpeg, abc_2.jpeg, xyz_0.jpeg, mno_0.jpeg, mno_1.jpeg , where, species abc, xyz and mno have 3, 1 and 2 images respectively.
    - The program also makes two lists / new columns containing details of species for which no urls for images were found on inaturalist.nz and the details of number of images found for each species.
    - Results : a total of 1776 images were downloaded (of 733 distinct species out of 1500). A POINT OF CONTENTION as  inaturalist.nz webpage shows a count of only 550 research grade pictures of introduced species. (NEEDS RECONCILIATION or further investigation).


### Idea of the program
    - The program looks up the search page of inaturalist.nz for each insect and further looks for 'browse_photos' page of each insects using the research grade filter. From this web page, urls for large images are selected out of the html text of the page.
    - further, the images are downloaded with a delay of one second each.

In [14]:
import pandas as pd
from selenium import webdriver

In [15]:
import requests
from bs4 import BeautifulSoup as bs
import urllib
import time 
import os

## Getting the Names from Exotic Insects List

In [ ]:
# Original GBIF Exotic data in excel format. Reading excel file using pandas 
data = pd.read_excel("specieslist.xlsx")
data.columns

In [18]:
list_species = data['Original species name']
list_species = list_species.unique()

In [19]:
names = []
for name in list_species: 
    first = name.split()[0]
    second = name.split()[1]
    new_name = first + "-" + second
    names.append(new_name)

##  Obtaining the urls of large images from the browse_photos webpage of inaturalist using the 'research' grade filter, based on the Exotic Species list

In [21]:
driver = webdriver.Firefox(executable_path=r"path/geckodriver-v0.23.0-win64/geckodriver.exe")

In [24]:
urls_for_images = []
searchUrl_image_link_list = []
for name in names:
    
    url = "https://inaturalist.nz/taxa/search?q="+name.lower()
    driver.get(url)
    
    html = driver.page_source
    soup = bs(html, "lxml")
    links  = soup.find_all("a")

    
    searchUrl= []
    for link in links:
        name_with_code = link.get('href')
        if name_with_code.endswith(name):
            searchUrl.append(name_with_code)
            
            #print(name_with_code)
        
    
    if len(searchUrl) >0:
        #searchUrl_image_link_list.append('https://inaturalist.nz'+searchUrl[0]+'/browse_photos')
        searchUrl_image_link_list.append('https://inaturalist.nz'+searchUrl[0]+'/browse_photos?quality_grade=research&photo_license=any')
    else:
        searchUrl_image_link_list.append('NA')
        #print('https://inaturalist.nz'+searchUrl[0]+'/browse_photos')

In [25]:
searchUrl_image_link_list[:7]

['https://inaturalist.nz/taxa/122094-Abgrallaspis-cyanophylli/browse_photos',
 'NA',
 'https://inaturalist.nz/taxa/384843-Acantholybas-brunneus/browse_photos',
 'https://inaturalist.nz/taxa/212223-Acanthoscelides-obtectus/browse_photos',
 'https://inaturalist.nz/taxa/154814-Acanthuchus-trispinifer/browse_photos',
 'https://inaturalist.nz/taxa/384922-Achilus-flammeus/browse_photos',
 'https://inaturalist.nz/taxa/362715-Achroia-grisella/browse_photos']

In [26]:
exotic_url_df = pd.DataFrame({"original_species_name":list_species, "url":searchUrl_image_link_list})
exotic_url_df.head(5)

,original_species_name,url
0,Abgrallaspis cyanophylli,https://inaturalist.nz/taxa/122094-Abgrallaspi...
1,Acallopais rudis,NA
2,Acantholybas brunneus,https://inaturalist.nz/taxa/384843-Acantholyba...
3,Acanthoscelides obtectus,https://inaturalist.nz/taxa/212223-Acanthoscel...
4,Acanthuchus trispinifer,https://inaturalist.nz/taxa/154814-Acanthuchus...


In [30]:
exotic_url_df[0:4]

,original_species_name,url
0,Abgrallaspis cyanophylli,https://inaturalist.nz/taxa/122094-Abgrallaspi...
1,Acallopais rudis,NA
2,Acantholybas brunneus,https://inaturalist.nz/taxa/384843-Acantholyba...
3,Acanthoscelides obtectus,https://inaturalist.nz/taxa/212223-Acanthoscel...


### Function for downloading the images

In [28]:
# import time
# import random

# path = "T:/projects/2018/SCION/Stream 2/190113_i_nat_images"
# def get_images(name, url_list):
    
#     for i in range(len(url_list)):
#         seconds = random.randint(1,10)    
          # time.sleep(seconds)
#         # NOTE the stream=True parameter
#         r = requests.get(url_list[i], stream=True)
#         with open("{}/{}_{}.jpeg".format(path, name, i), 'wb') as f:
#             for chunk in r.iter_content(chunk_size=1024): 
#                 if chunk: # filter out keep-alive new chunks
#                     f.write(chunk)

In [ ]:
import time
url_availablity = []
image_availablity = []

text_browse_photos = []

for i in range(len(exotic_url_df)):
    time.sleep(1)
    name = exotic_url_df["original_species_name"][i]
    print(i)
    url = exotic_url_df["url"][i]
    #print(url)
    if url != 'NA':
        #print(url)
        driver.get(url)

        html = driver.page_source
        soup = bs(html, "lxml")
        text  = soup.find_all("script")
        #print(text)
        #web_text.append(text)
        text_browse_photos.append(text)
        text_list = str([text]).split(',')
        url_list = []
        for item in text_list:
                       
            item = item.strip('"')
            if item.startswith("large_url"): # or item.startswith("url")or item.startswith("medium_url") :
                item = item.split(':"')
                link_large = item[1].strip('"}')
                #print(link_large)
                url_list.append(link_large)
                
        url_availablity.append(1)
        
        if len(url_list) > 0:
            #get_images(name, url_list) 
            image_availablity.append(len(url_list))
            
        else:
            image_availablity.append(0)
            
    else:
        text_browse_photos.append("NA")
        url_availablity.append(0)
        image_availablity.append(0)        
        

### Getting the Images

In [37]:
exotic_url_df["url_availablity"] = url_availablity

In [38]:
exotic_url_df["image_availablity"] = image_availablity

In [54]:
exotic_species_with_inat_pic = len(exotic_url_df[exotic_url_df.image_availablity != 0])
exotic_species_with_inat_pic

733

In [55]:
exotic_species_with_photos_url = len(exotic_url_df[exotic_url_df.url_availablity != 0])
exotic_species_with_photos_url

1366

In [56]:
#Total photos downloaded : 1776/ good photos = 1704

sum(image_availablity)

1776

#### List of Good Photos

In [61]:
subdir = 'T:/projects/2018/SCION/Stream 2/190113_GOOD_i_nat_images'

species_name = {}

for images in os.listdir(subdir):
    
    new = images.split(' ')
    #print(new[0] + ' ' + new[1])
    first_name = new[0] 
    second_name = new[1].split('_')[0]
    insect_name = first_name + ' ' + second_name
    if insect_name not in species_name:
        species_name[insect_name] = 1
    
    else:
        species_name[insect_name] += 1
    

In [ ]:
insects_with_inat_photos = list(species_name.keys())
insects_with_inat_photos

In [66]:
good_photo_details = pd.DataFrame({'insects_with_inat_photos' : insects_with_inat_photos, "number_of_photos_each_insect":number_of_photos_each_insect})

In [67]:
good_photo_details.head(5)

,insects_with_inat_photos,number_of_photos_each_insect
0,Acantholybas brunneus,1
1,Acanthoscelides obtectus,4
2,Acanthuchus trispinifer,1
3,Achilus flammeus,1
4,Achroia grisella,1


In [68]:
len(good_photo_details)

716

In [69]:
sum(number_of_photos_each_insect)

1698

### Dataframe with HTML text for later use if required.

In [42]:
#text_browse_photos[:1]

In [43]:
exotic_url_df_with_text = exotic_url_df

In [44]:
exotic_url_df_with_text["text_browse_photo_page"] =text_browse_photos

In [ ]:
#exotic_url_df.to_csv("T:/projects/2018/SCION/Stream 2/190114_LIST_OF_INAT_IMAGES_WITH TEXT.csv")

In [45]:
exotic_url_df_with_text.to_csv("T:/projects/2018/SCION/Stream 2/190114_LIST_OF_INAT_IMAGES_WITH TEXT.csv")

In [47]:
exotic_url_df_with_text.head(5)

,original_species_name,url,url_availablity,image_availablity,text_browse_photo_page
0,Abgrallaspis cyanophylli,https://inaturalist.nz/taxa/122094-Abgrallaspi...,1,0,"[<script src=""https://js-agent.newrelic.com/nr..."
1,Acallopais rudis,NA,0,0,NA
2,Acantholybas brunneus,https://inaturalist.nz/taxa/384843-Acantholyba...,1,1,"[<script src=""https://js-agent.newrelic.com/nr..."
3,Acanthoscelides obtectus,https://inaturalist.nz/taxa/212223-Acanthoscel...,1,5,"[<script src=""https://js-agent.newrelic.com/nr..."
4,Acanthuchus trispinifer,https://inaturalist.nz/taxa/154814-Acanthuchus...,1,1,"[<script src=""https://js-agent.newrelic.com/nr..."


## THE END 